Import and assign dataframe. 

In [1]:
import numpy as np, pandas as pd, plotly.express as px, plotly.graph_objects as go

ameo_df = pd.read_csv("AMEO_full_data.csv")
solar_daily_df = pd.read_csv("solar_coverage_daily_data.csv")
solar_monthly_df = pd.read_csv("solar_coverage_monthly_data.csv")

Check infos

In [2]:
print(ameo_df.info())
print('\n')
print(ameo_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497023 entries, 0 to 497022
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   REGION          497023 non-null  object 
 1   SETTLEMENTDATE  497023 non-null  object 
 2   TOTALDEMAND     497023 non-null  float64
 3   RRP             497023 non-null  float64
dtypes: float64(2), object(2)
memory usage: 15.2+ MB
None


  REGION       SETTLEMENTDATE  TOTALDEMAND    RRP
0   NSW1  1998-12-07 02:00:00   3294.38000   8.01
1   NSW1  1998-12-07 02:30:00   5337.07333  11.16
2   NSW1  1998-12-07 03:00:00   5296.28500  13.52
3   NSW1  1998-12-07 03:30:00   5265.64000  12.52
4   NSW1  1998-12-07 04:00:00   5330.04833  13.01


Notice on only 11464 non-null count in "Daily global solar exposure (MJ/m*m)" when there are 11938 entries. 

In [3]:
print(solar_daily_df.info())
print('\n')
print(solar_daily_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11938 entries, 0 to 11937
Data columns (total 6 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Product code                          11938 non-null  object 
 1   Bureau of Meteorology station number  11938 non-null  int64  
 2   Year                                  11938 non-null  int64  
 3   Month                                 11938 non-null  int64  
 4   Day                                   11938 non-null  int64  
 5   Daily global solar exposure (MJ/m*m)  11464 non-null  float64
dtypes: float64(1), int64(4), object(1)
memory usage: 559.7+ KB
None


  Product code  Bureau of Meteorology station number  Year  Month  Day  \
0   IDCJAC0016                                 70247  1990      1    1   
1   IDCJAC0016                                 70247  1990      1    2   
2   IDCJAC0016                                 70247  1990     

In [4]:
# print(solar_monthly_df.info())
# print('\n')
# print(solar_monthly_df.head())

Data cleaning

In [5]:
######## Clean solar_daily ########
 
# Fill null values.
solar_daily = solar_daily_df.drop(['Product code', 'Bureau of Meteorology station number', 'Year', 'Month', 'Day'], axis=1).ffill() 

# Combine date data into solar_daily
date = [(pd.to_datetime('%s-%s-%s' % (solar_daily_df.Year[i], solar_daily_df.Month[i], solar_daily_df.Day[i]))) for i in range(len(solar_daily_df))]
solar_daily['Date'] = date

# Convert from solar_daily to solar_monthly sum all monthly data
solar_daily.set_index('Date', inplace=True)
solar_daily['Date'] = pd.to_datetime(solar_daily.index)
solar_monthly_primal = solar_daily.resample('1M').sum()
solar_monthly = solar_monthly_primal

######## Clean ameo ########

# Convert to datetime
ameo_df['SETTLEMENTDATE'] = ameo_df["SETTLEMENTDATE"].astype('datetime64[ns]')

# Convert from ameo_daily to ameo_monthly sum all monthly data, and rename column dame
ameo_df.set_index('SETTLEMENTDATE', inplace=True)
ameo_df["SETTLEMENTDATE"] = pd.to_datetime(ameo_df.index)
ameo_monthly = ameo_df.resample('1M').sum()[:-1].reset_index()[:-32].rename(columns = {'SETTLEMENTDATE' : 'Date'})

Notice in our findings, we see 393 entries compare to original solar_monthly_df as 392. Since the latest september is not complete, we can remove the last row. 

In [6]:
# Remove september data and rename column name from daily to monthly
solar_monthly = solar_monthly_primal[:-33].rename(columns = {'Daily global solar exposure (MJ/m*m)' : 'Monthly_solar_exposure(MJ/m*m)'}).reset_index() 

In [7]:
fig1 = px.line(solar_monthly, x="Date", y="Monthly_solar_exposure(MJ/m*m)").show()
fig2 = px.line(ameo_monthly, x="Date", y="TOTALDEMAND").show()

Code below is average out all the month-to-month data throughout the years.

In [8]:
total_solar = solar_monthly.groupby(solar_monthly["Date"].dt.month)["Monthly_solar_exposure(MJ/m*m)"].mean().reset_index()
total_ameo_demand = ameo_monthly.groupby(ameo_monthly["Date"].dt.month)["TOTALDEMAND"].mean().reset_index()
total_ameo_rrp = ameo_monthly.groupby(ameo_monthly["Date"].dt.month)["RRP"].mean().reset_index()

In [9]:
fig_solar = px.line(x=total_solar["Date"], y=total_solar["Monthly_solar_exposure(MJ/m*m)"], labels={'x' : 'Month', 'y' : "Monthly_solar_exposure(MJ/m*m)"}, title="Average solar exposure value in each months from 1990 to 2019").show()
fig_ameo_demand = px.line(x=total_ameo_demand["Date"], y=total_ameo_demand["TOTALDEMAND"], labels={'x' : 'Month', 'y' : '$/MWh'}, title="Average total demand value in each months from 1998 to 2019").show()
fig_ameo_rrp = px.line(x=total_ameo_rrp["Date"], y=total_ameo_rrp["RRP"], labels={'x' : 'Month', 'y' : '$'}, title="Average RRP value in each months from 1998 to 2019").show()